## Простейшая модель 

### План

- преобразовать текст в ветор частей речи (+ падеж + спряжение там где это актуально)
- построить модель на би и триграммах

Некоторые идеи созвучны статье (О.В. КУКУШКИНА, А.А.ПОЛИКАРПОВ, Д.В. ХМЕЛЁВ - ОПРЕДЕЛЕНИЕ АВТОРСТВА ТЕКСТА С ИСПОЛЬЗОВАНИЕМ БУКВЕННОЙ И ГРАММАТИЧЕСКОЙ ИНФОРМАЦИИ)

https://www.philol.msu.ru/~lex/khmelev/published/gramcodes/gramcodeswin.html#tth_sEc2

### Загрузка данных

In [1]:
import wandb

# S3
import boto3
import pickle
from io import BytesIO, StringIO

import pandas as pd
import re
import nltk

from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from nltk import ngrams
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [3]:
BUCKET_NAME = "mlds23-authorship-identification"
DATA_DIR = "splitted_data/"
DATA_FILE_NAME = 'splitted_df.csv'
MODELS_DIR = 'models/'
RANDOM_STATE = 12345
RUN_NAME = 'dmitry-20-02-24-exp-1'
WANDB_NOTEBOOK_NAME = 'Base_model_(D.Shiltsov)'

In [4]:
label2name = {
    0: 'А. Пушкин',
    1: 'Д. Мамин-Сибиряк',
    2: 'И. Тургенев',
    3: 'А. Чехов',
    4: 'Н. Гоголь',
    5: 'И. Бунин',
    6: 'А. Куприн',
    7: 'А. Платонов',
    8: 'В. Гаршин',
    9: 'Ф. Достоевский'
}

In [5]:
session = boto3.session.Session()

#splitted_data/splitted_df_1000.csv
#splitted_data/splitted_df_2000.csv
#splitted_data/splitted_df_3000.csv

s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJErlaldUmioGbHQSqJ70MR',
    aws_secret_access_key='YCPSba_JgloNYSNWcnKO2CYCEB8PFR1Iwgr2jIUy',
    region_name='ru-cental1'
)

BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
#s3.download_file(Filename="data/" + FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)
#загружает в локальную директорию, потом отдельно надо считывать
for i in range (1,4):
    FNAME = f'splitted_df_{i}000.csv'
#    s3.download_file(Filename="data/" + FNAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FNAME)

In [6]:
df = pd.read_csv('./data/splitted_df_3000.csv')
print(df.shape)
df.head()

(2193, 3)


target                                               text  \
0  author_id_06  Узкими горными тропинками , от одного дачного ...   
1  author_id_06  в вечных скитаниях : пуделя Арто и маленького ...   
2  author_id_06  , когда весь Крым наполняется нарядной , богат...   
3  author_id_06  махали на них с балконов руками , в третьих пр...   
4  author_id_06  запыленных кипарисов , похожих на длинные черн...   

                                         book  
0  raw_data/aleksandr_kuprin_belyj_pudel'.txt  
1  raw_data/aleksandr_kuprin_belyj_pudel'.txt  
2  raw_data/aleksandr_kuprin_belyj_pudel'.txt  
3  raw_data/aleksandr_kuprin_belyj_pudel'.txt  
4  raw_data/aleksandr_kuprin_belyj_pudel'.txt

In [7]:
# сохранение признаков и целевого признака в отдельные переменные
X = df['text']
y = df['target'].map(lambda x: int(x[-2:]))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=.3, random_state=42)


## Кодирование признаков

- исходим из предположения что знаки препинания являются важными признаками, поэтому закодируем некоторые из них в слова
- закодируем слова как часть речи, для частей речи имеющих падеж добавим падеж (сущ, прил, местоимение), для частей речи имеющих спряжение добавим спряжение. Формат - ЧАСТЬРЕЧИ_ПАДЕЖ 

In [82]:
def clean_sentence(sentence):
    sentence = re.sub(r"[^а-яА-ЯёЁ \-\"!'(),.:;?]", '', sentence) 
    return sentence

# тест
clean_sentence("как-то - 'рано' 'Marta'? пела: лЕСОМ, * &нифига(она) не ела")

"как-то - 'рано' ''? пела: лЕСОМ,  нифига(она) не ела"

In [83]:
def make_punkt(sentence):
    repl = [('.',' POINT '), (',',' COMMA '), ('?',' QST '), ('!',' EXCL '), 
            (':',' COLON '), (';',' SEMICOL '), (',',' DASH '),]
    for p, r in repl:
        sentence = sentence.replace(p,r)
    sentence = re.sub(r"\s?-\s|\s-\s?", ' DASH ', sentence) # не трогать тире в слове (как-то)
        
    return sentence
        
# тест    
make_punkt("как-то - 'рано' 'Marta'? пела: лЕСОМ, * &нифига(она) не ела")    

"как-то DASH 'рано' 'Marta' QST  пела COLON  лЕСОМ COMMA  * &нифига(она) не ела"

In [84]:
mystem_analyzer = Mystem()

def make_grams(sentence):
    morph = mystem_analyzer.analyze(sentence)
    
    ret = []
    for lex in morph:
        if lex['text'] in ['POINT', 'COMMA', 'QST', 'EXCL', 'COLON', 'SEMICOL', 'DASH']:
            ret.append(lex['text'])
            continue
        
        try:
            if 'analysis' in lex.keys() and 'gr' in lex['analysis'][0].keys():
                ret.append(lex['analysis'][0]['gr'])
        except:
            # встретил что-то непотребное в стиле ру-ру-ру
            pass
    return ' '.join(ret)

# тест
make_grams("и POINT столу но и тёплым снегом")

'CONJ= POINT S,муж,неод=дат,ед CONJ= CONJ= A=(дат,мн,полн|твор,ед,полн,муж|твор,ед,полн,сред) S,муж,неод=твор,ед'

In [85]:
def make_grams_brief(sentence):
    morph = mystem_analyzer.analyze(sentence)
    
    ret = []
    for lex in morph:
        if lex['text'] in ['POINT', 'COMMA', 'QST', 'EXCL', 'COLON', 'SEMICOL', 'DASH']:
            ret.append(lex['text'])
            continue
        
        try:
            if 'analysis' in lex.keys() and 'gr' in lex['analysis'][0].keys():
                ret.append(lex['analysis'][0]['gr'].split('=')[0])
        except:
            # встретил что-то непотребное в стиле ру-ру-ру
            pass
    return ' '.join(ret)

# тест
make_grams_brief("и POINT столу но и тёплым снегом")

'CONJ POINT S,муж,неод CONJ CONJ A S,муж,неод'

In [86]:
def prepare_text(Text_corp):
    res = []
    for text in Text_corp:
        text = clean_sentence(text)
        text = make_punkt(text)
        text = make_grams(text)
        res.append(text)
    return res

X_tr = prepare_text(X_train)
X_ts = prepare_text(X_test)


## Подготовка BOW и обучение простейшей линейной модели



In [87]:
wandb.login()
wandb.init(project="authorship_identification", name='LR_BOW', entity="mlds23_ai", tags=['baseline_grams'])

# логирование гиперпараметров данных, предобработки и модели
wandb.config['dataset'] = DATA_FILE_NAME

wandb.config['preprocessing'] = {
    'split_size':3000,
    'clean_sentence': True,
    'make_punkt': True,
    'make_grams': True,
    'make_grams_brief': False,
    'prepare_text': True
}

wandb.config['embedding'] = {
    'embedding_type': 'CountVectorizer ngram(2,4)'    
}

wandb.config['classifier'] = {
    'classifier_name': 'LogisticRegression',
    'max_iter': 1000,
}

wandb.config['evaluation'] = {
    'test_size': 0.25,
}

In [93]:
myvec = CountVectorizer(ngram_range=(1, 4), 
                        stop_words=None, lowercase=False, 
                        token_pattern=r"(?u)\b[a-zA-Zа-яА-ЯёЁ0-9,=90|\-]+\b")
X_tr_bow  = myvec.fit_transform(X_tr)
X_ts_bow  = myvec.transform(X_ts)
#myvec.vocabulary_.items()

In [94]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_tr_bow, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [95]:
pred = clf.predict(X_ts_bow)
cls_report = classification_report(pred, y_test)
print(cls_report)

              precision    recall  f1-score   support

author_id_00       0.90      0.88      0.89        89
author_id_01       0.88      0.88      0.88        16
author_id_02       0.94      0.84      0.89       159
author_id_03       0.77      0.80      0.79        51
author_id_04       0.87      0.90      0.88        72
author_id_05       0.80      0.78      0.79        36
author_id_06       0.79      0.76      0.77        54
author_id_07       0.74      1.00      0.85        17
author_id_08       0.62      0.84      0.71        25
author_id_09       0.90      0.91      0.91       139

    accuracy                           0.86       658
   macro avg       0.82      0.86      0.84       658
weighted avg       0.87      0.86      0.86       658



In [91]:
# логирование метрик
metrics_train = dict()
metrics_train['accuracy'] = accuracy_score(y_test, pred)
metrics_train['f1_macro'] = f1_score(y_test, pred, average='macro')
metrics_train['f1_weighted'] =  f1_score(y_test, pred, average='weighted')

wandb.log({
    'test': metrics_train
    })

wandb.log({
    'conf_mat_test': wandb.plot.confusion_matrix(probs=None, 
                                                 y_true=y_test.tolist(), preds=pred)   
})


In [92]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


### Лес случайностей (Random forest)

In [100]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200, max_depth=2000, n_jobs=-1)
rfc.fit(X_tr_bow, y_train)

RandomForestClassifier(max_depth=2000, n_estimators=200, n_jobs=-1)

In [101]:
pred = rfc.predict(X_ts_bow)
cls_report = classification_report(pred, y_test)
print(cls_report)

              precision    recall  f1-score   support

author_id_00       0.29      0.96      0.44        26
author_id_01       0.00      0.00      0.00         0
author_id_02       0.96      0.35      0.51       397
author_id_03       0.00      0.00      0.00         0
author_id_04       0.29      0.88      0.44        25
author_id_05       0.14      1.00      0.25         5
author_id_06       0.27      0.70      0.39        20
author_id_07       0.00      0.00      0.00         1
author_id_08       0.00      0.00      0.00         0
author_id_09       0.80      0.61      0.70       184

    accuracy                           0.48       658
   macro avg       0.28      0.45      0.27       658
weighted avg       0.84      0.48      0.55       658



/home/sda/Work/HSE-MLHS/Coursework/nlpenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sda/Work/HSE-MLHS/Coursework/nlpenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sda/Work/HSE-MLHS/Coursework/nlpenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

### Попробуем улучшить результат сократив информацию по частям речи

In [72]:
wandb.login()
wandb.init(project="authorship_identification", name='LR_BOW', entity="mlds23_ai", tags=['baseline_grams_reduced'])

# логирование гиперпараметров данных, предобработки и модели
wandb.config['dataset'] = DATA_FILE_NAME

wandb.config['preprocessing'] = {
    'split_size':3000,
    'clean_sentence': True,
    'make_punkt': True,
    'make_grams': True,
    'make_grams_brief': False,
    'prepare_text': True
}

wandb.config['embedding'] = {
    'embedding_type': 'CountVectorizer'    
}

wandb.config['classifier'] = {
    'classifier_name': 'LogisticRegression',
    'max_iter': 1000,
}

wandb.config['evaluation'] = {
    'test_size': 0.25,
}

In [73]:
def prepare_text(Text_corp):
    res = []
    for text in Text_corp:
        text = clean_sentence(text)
        text = make_punkt(text)
        text = make_grams_brief(text)
        res.append(text)
    return res

X_trb = prepare_text(X_train)
X_tsb = prepare_text(X_test)


In [74]:
myvec = CountVectorizer(ngram_range=(1, 4), 
                        stop_words=None, lowercase=False, 
                        token_pattern=r"(?u)\b[a-zA-Zа-яА-ЯёЁ0-9,=90|\-]+\b")
X_trb_bow  = myvec.fit_transform(X_trb)
X_tsb_bow  = myvec.transform(X_tsb)
#myvec.vocabulary_.items()

In [75]:
clf = LogisticRegression(random_state=42, max_iter=1500)
clf.fit(X_trb_bow, y_train)

LogisticRegression(max_iter=1500, random_state=42)

In [76]:
pred = clf.predict(X_tsb_bow)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

author_id_00       0.87      0.92      0.89        83
author_id_01       0.94      0.79      0.86        19
author_id_02       0.91      0.88      0.89       147
author_id_03       0.79      0.76      0.78        55
author_id_04       0.93      0.83      0.88        84
author_id_05       0.86      0.77      0.81        39
author_id_06       0.77      0.89      0.82        45
author_id_07       0.87      0.95      0.91        21
author_id_08       0.62      0.72      0.67        29
author_id_09       0.87      0.90      0.89       136

    accuracy                           0.86       658
   macro avg       0.84      0.84      0.84       658
weighted avg       0.86      0.86      0.86       658



In [78]:
# логирование метрик
metrics_train = dict()
metrics_train['accuracy'] = accuracy_score(y_test, pred)
metrics_train['f1_macro'] = f1_score(y_test, pred, average='macro')
metrics_train['f1_weighted'] =  f1_score(y_test, pred, average='weighted')

wandb.log({
    'test': metrics_train
    })

wandb.log({
    'conf_mat_test': wandb.plot.confusion_matrix(probs=None, 
                                                 y_true=y_test.tolist(), preds=pred)   
})

wandb.finish()

Error: You must call wandb.init() before wandb.log()